In [ ]:
import re

import json

import os

In [18]:
DATA_PATH="data/"

In [19]:
json_normas_path = DATA_PATH + 'normas_full.json'

with open(json_normas_path, 'r') as file:
    normas_json = json.load(file)

normas = list(normas_json.values())

In [22]:
normas_json.keys()

dict_keys(['Ato Declaratório Cosar nº 47.txt', 'Ato Declaratório Interpretativo RFB nº 1.txt', 'Ato Declaratório Interpretativo SRF nº 2.txt', 'Ato Declaratório Interpretativo SRF nº 24.txt', 'Ato Declaratório Interpretativo SRF nº 5.txt', 'Ato Declaratório PGFN nº 1.txt', 'Ato Declaratório PGFN nº 13.txt', 'Ato Declaratório PGFN nº 14.txt', 'Ato Declaratório PGFN nº 2.txt', 'Ato Declaratório PGFN nº 3.txt', 'Ato Declaratório PGFN nº 4.txt', 'Ato Declaratório PGFN nº 5.txt', 'Ato Declaratório PGFN nº 6 DE 2006.txt', 'Ato Declaratório PGFN nº 6 DE 2008.txt', 'Ato Declaratório PGFN nº 9.txt', 'Ato Declaratório RFB nº 16.txt', 'Ato Declaratório RFB nº 18.txt', 'Ato Declaratório RFB nº 3.txt', 'Ato Declaratório SRF nº 14.txt', 'Ato Declaratório SRF nº 16.txt', 'Ato Declaratório SRF nº 22.txt', 'Ato Declaratório SRF nº 26.txt', 'Ato Declaratório SRF nº 28.txt', 'Ato Declaratório SRF nº 3.txt', 'Ato Declaratório SRF nº 48.txt', 'Ato Declaratório SRF nº 7.txt', 'Ato Declaratório SRF nº 8.txt'

In [23]:
normas_json['RIR2018.txt']

'Presidência da República\nSecretaria-Geral\nSubchefia para Assuntos Jurídicos\nDECRETO Nº 9.580, DE 22 DE NOVEMBRO DE 2018\n \nRegulamenta a tributação, a fiscalização, a arrecadação e a\nadministração do Imposto sobre a Renda e Proventos de Qualquer\nNatureza.\nO PRESIDENTE DA REPÚBLICA , no uso da atribuição que lhe confere o art. 84, caput , inciso IV, da Constituição, e tendo em vista o disposto\nnas leis do Imposto sobre a Renda e Proventos de Qualquer Natureza,\nDECRETA:\nArt. 1º Fica aprovado o Regulamento do Imposto sobre a Renda e Proventos de Qualquer Natureza, anexo a este Decreto.\nArt. 2º O Imposto sobre a Renda e Proventos de Qualquer Natureza será cobrado, fiscalizado, arrecadado e administrado em conformidade com o\ndisposto neste Regulamento.\nArt. 3º Este Regulamento consolida a legislação referente ao Imposto sobre a Renda e Proventos de Qualquer Natureza publicada até 31 de\ndezembro de 2016.\nArt. 4º Fica revogado o Decreto nº 3.000, de 26 de março de 1999 .\nArt.

## Creating an hierarchical splitter

Using as base the code in https://github.com/langchain-ai/langchain/blob/master/libs/text-splitters/langchain_text_splitters/character.py

In [47]:
from typing import Any, List, Literal, Optional, Union
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [111]:
class HierachicalRecursiveCharacterTextSplitter(RecursiveCharacterTextSplitter):

    def _split_text_with_regex(
        self, text: str, separator: str
    ) -> List[str]:
        # Now that we have the separator, split the text

        separators = None
        
        if separator:
            if self._keep_separator:
                try:
                    # The parentheses in the pattern keep the delimiters in the result.
                    _splits = re.split(f"({separator})", text)
                    splits = (
                        ([_splits[i] + _splits[i + 1] for i in range(0, len(_splits) - 1, 2)])
                        if self._keep_separator == "end"
                        else ([_splits[i] + _splits[i + 1] for i in range(1, len(_splits), 2)])
                    )
                    if len(_splits) % 2 == 0:
                        splits += _splits[-1:]
                    splits = (
                        (splits + [_splits[-1]])
                        if self._keep_separator == "end"
                        else ([_splits[0]] + splits)
                    )

                    separators = ["preamble"] + [_splits[i].strip("\n").strip() for i in range(1, len(_splits), 2)]
                except IndexError as e:
                    print(e)
                    print(_splits)
            else:
                splits = re.split(separator, text)
        else:
            splits = list(text)
        return [s for s in splits if s != ""], separators



    def _split_text(self, text: str, separators: List[str]) -> List[str]:
        """Split incoming text and return chunks."""
        final_chunks = {}
        # Get appropriate separator to use
        separator = separators[-1]
        new_separators = []
        for i, _s in enumerate(separators):
            _separator = _s if self._is_separator_regex else re.escape(_s)
            if _s == "":
                separator = _s
                break
            if re.search(_separator, text):
                separator = _s
                new_separators = separators[i + 1 :]
                break

        _separator = separator if self._is_separator_regex else re.escape(separator)
        splits, separators = self._split_text_with_regex(text, _separator)

        # Does not merge small chunks to respect hierarchy

        if not new_separators:
            final_chunks = dict(zip(separators, splits))
        else:
            for i, s in enumerate(splits):
                other_info = self._split_text(s, new_separators)
                final_chunks[separators[i]] = other_info

        return final_chunks    

In [155]:
spliting_hierarchy=[
    "\nLIVRO [IVX]+",
    "\nTÍTULO [IVX]+",
    "\nCAPÍTULO [IVX]+|\nCAPÍTULO ÚNICO",
    "\nSeção [IVX]+",
    "\nSubseção .+",
    "\nArt\. \d+[º\. ]*",
    # "\n§ \d+[ºº\. ]*"
]

In [156]:
text_splitter = HierachicalRecursiveCharacterTextSplitter(
    separators=spliting_hierarchy,
    is_separator_regex=True,
    chunk_size=1000,  # Tamanho máximo do chunk em caracteres
    chunk_overlap=100  # Sobreposição entre chunks para contexto
)

In [157]:
chunks = text_splitter.split_text(normas_json['RIR2018.txt'])

### Check the extracted data hierarchy

In [158]:
len(chunks)

5

In [159]:
chunks.keys()

dict_keys(['preamble', 'LIVRO I', 'LIVRO II', 'LIVRO III', 'LIVRO IV'])

In [137]:
chunks['preamble']

{'preamble': 'Presidência da República\nSecretaria-Geral\nSubchefia para Assuntos Jurídicos\nDECRETO Nº 9.580, DE 22 DE NOVEMBRO DE 2018\n \nRegulamenta a tributação, a fiscalização, a arrecadação e a\nadministração do Imposto sobre a Renda e Proventos de Qualquer\nNatureza.\nO PRESIDENTE DA REPÚBLICA , no uso da atribuição que lhe confere o art. 84, caput , inciso IV, da Constituição, e tendo em vista o disposto\nnas leis do Imposto sobre a Renda e Proventos de Qualquer Natureza,\nDECRETA:',
 'Art. 1º': '\nArt. 1º Fica aprovado o Regulamento do Imposto sobre a Renda e Proventos de Qualquer Natureza, anexo a este Decreto.',
 'Art. 2º': '\nArt. 2º O Imposto sobre a Renda e Proventos de Qualquer Natureza será cobrado, fiscalizado, arrecadado e administrado em conformidade com o\ndisposto neste Regulamento.',
 'Art. 3º': '\nArt. 3º Este Regulamento consolida a legislação referente ao Imposto sobre a Renda e Proventos de Qualquer Natureza publicada até 31 de\ndezembro de 2016.',
 'Art. 4º'

In [138]:
chunks["LIVRO I"].keys()

dict_keys(['preamble', 'TÍTULO I', 'TÍTULO II', 'TÍTULO III', 'TÍTULO IV', 'TÍTULO V', 'TÍTULO VI', 'TÍTULO VII', 'TÍTULO VIII', 'TÍTULO IX', 'TÍTULO X'])

In [139]:
chunks["LIVRO I"]['TÍTULO I'].keys()

dict_keys(['preamble', 'CAPÍTULO I', 'CAPÍTULO II', 'CAPÍTULO III'])

In [140]:
chunks["LIVRO I"]['TÍTULO I']['preamble']

{'preamble': '\nTÍTULO I\nDOS CONTRIBUINTES E RESPONSÁVEIS'}

In [141]:
chunks["LIVRO I"]['TÍTULO I']['CAPÍTULO II'].keys()

dict_keys(['preamble', 'Seção I', 'Seção II', 'Seção III', 'Seção IV', 'Seção V', 'Seção VI', 'Seção VII'])

In [142]:
chunks["LIVRO I"]['TÍTULO I']['CAPÍTULO II']['Seção I'].keys()

dict_keys(['preamble', 'Subseção única'])

In [146]:
chunks["LIVRO I"]['TÍTULO I']['CAPÍTULO II']['Seção I']['preamble'].keys()

dict_keys(['preamble', 'Art. 3º'])

In [149]:
chunks["LIVRO I"]['TÍTULO I']['CAPÍTULO II']['Seção I']['preamble']['preamble']

'\nSeção I\nDos menores e de outros incapazes'

In [144]:
chunks["LIVRO I"]['TÍTULO I']['CAPÍTULO II']['Seção II'].keys()

dict_keys(['preamble', 'Subseção I', 'Subseção II', 'Subseção III'])

In [147]:
chunks["LIVRO I"]['TÍTULO I']['CAPÍTULO II']['Seção II']['Subseção I']

{'preamble': '\nSubseção I\nDa tributação em separado',
 'Art. 6º': '\nArt. 6º Cada cônjuge deverá incluir, em sua declaração, a totalidade dos rendimentos próprios e a metade dos rendimentos produzidos pelos\nbens comuns.\n§ 1º O imposto sobre a renda pago ou retido na fonte sobre os rendimentos produzidos pelos bens comuns deverá ser compensado na\ndeclaração, na proporção de cinquenta por cento para cada um dos cônjuges, independentemente de qual deles tenha sofrido a retenção ou efetuado o\nrecolhimento.\n§ 2º Na hipótese prevista no § 1º do art. 5º, o imposto sobre a renda pago ou retido na fonte será compensado na declaração, em sua totalidade,\npelo cônjuge que declarar os rendimentos, independentemente de qual deles tenha sofrido a retenção ou efetuado o recolhimento.\n§ 3º Os bens comuns deverão ser relacionados somente por um dos cônjuges, se ambos estiverem obrigados à apresentação da declaração, ou,\nobrigatoriamente, pelo cônjuge que houver apresentado a declaração, quando

In [160]:
chunks["LIVRO I"]['TÍTULO VI'].keys()

dict_keys(['preamble', 'CAPÍTULO ÚNICO'])

In [161]:
chunks["LIVRO I"]['TÍTULO VI']['preamble']

{'preamble': '\nTÍTULO VI\nDA BASE DE CÁLCULO DO IMPOSTO SOBRE A RENDA NA DECLARAÇÃO',
 'Art. 76.': '\nArt. 76. A base de cálculo do imposto sobre a renda devido no ano-calendário será a diferença entre as somas ( Lei Complementar nº 109, de\n2001, art. 69 ; Lei nº 9.250, de 1995, art. 8º ; Lei nº 9.532, de 1997, art. 11 ; Lei nº 12.024, de 2009, art. 3 º; e Medida Provisória nº 2.158-35, de 2001,\nart. 61 ):\nI - dos rendimentos percebidos durante o ano-calendário, exceto os isentos, os não tributáveis, os tributáveis exclusivamente na fonte e os\nsujeitos à tributação definitiva; e\nII - das deduções relativas ao somatório dos valores de que tratam o art. 68 , o art. 70 , o art. 72 ao art. 75 , e da quantia, por dependente, de:\na) R$ 1.808,28 (um mil, oitocentos e oito reais e vinte e oito centavos), para o ano-calendário de 2010;\nb) R$ 1.889,64 (um mil, oitocentos e oitenta e nove reais e sessenta e quatro centavos), para o ano-calendário de 2011;\nc) R$ 1.974,72 (um mil, novecent

In [170]:
chunks["LIVRO I"]['TÍTULO VI']['preamble']['preamble']

'\nTÍTULO VI\nDA BASE DE CÁLCULO DO IMPOSTO SOBRE A RENDA NA DECLARAÇÃO'

In [162]:
chunks["LIVRO I"]['TÍTULO VI']['CAPÍTULO ÚNICO']

{'preamble': '\nCAPÍTULO ÚNICO\nDO DESCONTO SIMPLIFICADO',
 'Art. 77.': '\nArt. 77. Independentemente do montante dos rendimentos tributáveis na declaração de ajuste anual, recebidos no ano-calendário, o contribuinte\npoderá optar por desconto simplificado, que substituirá todas as deduções admitidas na legislação, correspondente à dedução de vinte por cento do\nvalor dos rendimentos tributáveis na declaração de ajuste anual, dispensadas a comprovação da despesa e a indicação de sua espécie, limitada a ( Lei\nnº 9.250, de 1995, art. 10, caput ):\nI - R$ 13.317,09 (treze mil, trezentos e dezessete reais e nove centavos), para o ano-calendário de 2010;\nII - R$ 13.916,36 (treze mil, novecentos e dezesseis reais e trinta e seis centavos), para o ano-calendário de 2011;\nIII - R$ 14.542,60 (quatorze mil, quinhentos e quarenta e dois reais e sessenta centavos), para o ano-calendário de 2012;\nIV - R$ 15.197,02 (quinze mil, cento e noventa e sete reais e dois centavos), para o ano-calendário

In [164]:
chunks["LIVRO II"]['TÍTULO I'].keys()

dict_keys(['preamble', 'CAPÍTULO I', 'CAPÍTULO II', 'CAPÍTULO III'])

In [169]:
chunks["LIVRO II"]['TÍTULO I']['CAPÍTULO II'].keys()

dict_keys(['preamble', 'Seção I', 'Seção II'])

In [171]:
chunks["LIVRO II"]['TÍTULO I']['CAPÍTULO II']['Seção II']

{'preamble': {'preamble': '\nSeção II\nDas empresas individuais imobiliárias'},
 'Subseção I': {'preamble': '\nSubseção I\nDa caracterização\nIncorporação e loteamento',
  'Art. 163.': '\nArt. 163. Serão equiparadas às pessoas jurídicas, em relação às incorporações imobiliárias ou aos loteamentos com ou sem construção, cuja\ndocumentação seja arquivada no Registro Imobiliário, a partir de 1º de janeiro de 1975 ( Decreto-Lei nº 1.381, de 1974, art. 6º ):\nI - as pessoas físicas que, nos termos estabelecidos nos art. 29, art. 30 e art. 68 da Lei nº 4.591, de 1964, no Decreto-Lei nº 58, de 10 de\ndezembro de 1937 , no Decreto-Lei nº 271, de 28 de fevereiro de 1967 , ou na Lei nº 6.766, de 19 de dezembro de 1979 , assumirem a iniciativa e a\nresponsabilidade de incorporação ou loteamento em terrenos urbanos ou rurais; e\nII - os titulares de terrenos ou glebas de terra que, nos termos estabelecidos no § 1º do art. 31 da Lei nº 4.591, de 1964 , ou no art. 3º do\nDecreto-Lei nº 271, de 1967 

### Flatten the extracted hierarchy

In [208]:
def flatten_passages_hierarchy(passages_dictionary, current_path="", passages=[]):

    for item, value in passages_dictionary.items():
        item_path = current_path
        
        if item == "preamble":
            if type(value) == dict:
                flatten_passages_hierarchy(value, current_path, passages)
            # else:
            #     current_path += "_" + value
        else:
            if item_path != "":
                item_path += "_" + item
            else:
                item_path = item
            
            if type(value) == dict:
                flatten_passages_hierarchy(value, item_path, passages)
            else:
                passages.append({'path': item_path,
                                 'passage': value})        

In [209]:
passages = []
current_path = ""

In [210]:
flatten_passages_hierarchy(chunks, current_path, passages)

In [211]:
len(passages)

1032

In [215]:
passages[500]

{'path': 'LIVRO II_TÍTULO VIII_CAPÍTULO VI_Seção X_Art. 495.',
 'passage': '\nArt. 495. Até 31 de dezembro de 2018, a empresa construtora contratada para construir unidades habitacionais de valor de até R$ 100.000,00\n(cem mil reais) no âmbito do PMCMV, de que trata Lei nº 11.977, de 2009 , fica autorizada, em caráter opcional, a efetuar o pagamento unificado\nequivalente a um por cento da receita mensal auferida pelo contrato de construção ( Lei nº 12.024, de 2009, art. 2 º, caput ).\n§ 1º O pagamento mensal unificado de que trata o caput corresponderá aos seguintes tributos (Lei nº 12.024, de 2009, art. 2º, § 1º) :\nI - imposto sobre a renda das pessoas jurídicas;\nII - Contribuição para PIS/Pasep;\nIII - CSLL; e\nIV - Cofins.\n§ 2º O pagamento do imposto e das contribuições na forma prevista no caput será considerado definitivo e não gerará, em qualquer hipótese,\ndireito à restituição ou à compensação com o que for apurado pela construtora ( Lei nº 12.024, de 2009, art. 2º, § 2º ).

### Check passages tokens length

In [217]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("stjiris/bert-large-portuguese-cased-legal-tsdae")
model = AutoModel.from_pretrained("stjiris/bert-large-portuguese-cased-legal-tsdae")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

tokenizer_config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/678k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(29794, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, 

In [218]:
import statistics

def contar_tokens(chunk):
    tokens = tokenizer(chunk, return_tensors="pt", truncation=False)  # Tokeniza o texto sem truncar
    num_tokens = len(tokens['input_ids'][0])  # Conta o número de tokens
    return num_tokens

In [219]:
# Calcule o número de tokens para cada chunk
num_tokens_por_chunk = [contar_tokens(passage['passage']) for passage in passages]

# Cálculo das estatísticas
max_tokens = max(num_tokens_por_chunk)
min_tokens = min(num_tokens_por_chunk)
media_tokens = sum(num_tokens_por_chunk) / len(num_tokens_por_chunk)
mediana_tokens = statistics.median(num_tokens_por_chunk)

In [220]:
# Exibindo os resultados
print(f"Número máximo de tokens: {max_tokens}")
print(f"Número mínimo de tokens: {min_tokens}")
print(f"Média de tokens: {media_tokens}")
print(f"Mediana de tokens: {mediana_tokens}")

Número máximo de tokens: 7448
Número mínimo de tokens: 22
Média de tokens: 268.7073643410853
Mediana de tokens: 179.5


In [221]:
import numpy as np

In [222]:
np.argmax(num_tokens_por_chunk)

39

In [223]:
passages[39]

{'path': 'LIVRO I_TÍTULO IV_CAPÍTULO II_Art. 35.',
 'passage': '\nArt. 35. São isentos ou não tributáveis:\nI - os seguintes rendimentos originários do trabalho e assemelhados:\na) a alimentação, o transporte e os uniformes ou as vestimentas especiais de trabalho, fornecidos gratuitamente pelo empregador a seus\nempregados, ou a diferença entre o preço cobrado e o valor de mercado (Lei nº 7.713, de 1988, art. 6º, caput , inciso I);\nb) o auxílio-alimentação pago em pecúnia aos servidores públicos federais civis ativos da administração pública federal direta, autárquica e\nfundacional (Lei nº 8.460, de 17 de setembro de 1992, art. 22, § 1º e § 3º, alínea “b”);\nc) o valor recebido de pessoa jurídica de direito público a título de auxílio-moradia, não integrante da remuneração dos beneficiários, em\nsubstituição ao direito de uso de imóvel funcional (Medida Provisória nº 2.158-35, de 24 de agosto de 2001, art. 25);\nd) o auxílio-transporte em pecúnia, pago pela União, destinado ao custei

In [224]:
from scipy import stats

In [225]:
stats.describe(num_tokens_por_chunk)

DescribeResult(nobs=1032, minmax=(22, 7448), mean=268.7073643410853, variance=115634.0015751998, skewness=10.692553092268888, kurtosis=199.60361073650432)